<a href="https://colab.research.google.com/github/UetaKento/Editer_Like_Emacs_Python/blob/main/Editer_Like_Emacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# command "a"
def move_cursor_to_head():
    cursor["nth_char"] = 0

# command "e"
def move_cursor_to_end():
    cursor["nth_char"] = len(text[cursor["nth_line"]])-1

# command "p"
def move_cursor_up_and_to_head():
    if is_there_a_line("above"):
        move_cursor_by_one_position_to("above")
    move_cursor_to_head()
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# command "n"
def move_cursor_down_and_to_head():
    if is_there_a_line("below"):
        move_cursor_by_one_position_to("below")
    move_cursor_to_head()
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# command "f"
def move_cursor_to_right():
    if is_there_a_char_to("right"):
        move_cursor_by_one_position_to("right")
    elif is_there_a_line("below"): # 行末を指しており、かつ下にラインが存在するか確認しているか
        move_cursor_by_one_position_to("below")
        move_cursor_to_head()
    else:
        pass # 行末を指している場合かつ、下に行がない場合は何もしない。
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# command "b"
def move_cursor_to_left():
    if is_there_a_char_to("left"):
        move_cursor_by_one_position_to("left")
    elif is_there_a_line("above"): # 行頭を指しており、かつ上にラインが存在するか確認しているか
        move_cursor_by_one_position_to("above")
        move_cursor_to_end()
    else:
        pass

# command "d"
def delete_char_on_cursor():
    buf = ""
    if is_there_a_char_on_cursor():
        text[cursor["nth_line"]].pop(cursor["nth_char"]) # 二次元配列から文字列を削除、popのため文字列のシフトは必要ない。
    elif is_there_a_line("below"): #行末を指し、下に行があるときは、行末に下の行を連結する。その際に元の行の"\n"は削除
        buf = text.pop(cursor["nth_line"]+1)
        text[cursor["nth_line"]].pop(-1)
        text[cursor["nth_line"]].extend(buf)
    else:
        pass
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# command "k"
def delete_with_yanking():
    global buffer # グローバル変数のbufferを参照する
    if is_there_a_char_on_cursor():
        # 以下では「カーソルが指す文字を含めた右側すべての文字を切り取りそれをバッファに記録する。」
        buffer = text[cursor["nth_line"]][cursor["nth_char"]:-1]
        del text[cursor["nth_line"]][cursor["nth_char"]:-1]
        move_cursor_to_end()
    else:
        delete_char_on_cursor()
        buffer = "\n"
    assert buffer != "", "The buffer is empty!"
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# command "y"
def paste_buffer():
    buf = []
    if buffer == "":
        pass
    elif buffer == "\n": # バッファの内容が改行の場合、カーソルが指す文字の直前で改行を行う。カーソルは新しい行の先頭文字へ移動する。
        buf = text[cursor["nth_line"]][cursor["nth_char"]:]
        del text[cursor["nth_line"]][cursor["nth_char"]:-1]
        text.insert(cursor["nth_line"]+1, buf)
        move_cursor_by_one_position_to("below")
        move_cursor_to_head()
    else: # バッファの内容が文字列の場合、カーソルの位置にバッファの内容を挿入する。 カーソルの位置はもともと指していた文字または行末の場所へ移動する。
        text[cursor["nth_line"]][cursor["nth_char"]:cursor["nth_char"]] = buffer
        cursor["nth_char"] += len(buffer)
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# movementに関する補助関数たち
# カーソルの右または左に文字があるか確認する。
def is_there_a_char_to(direction):
    if direction.lower() == "right":
        if len(text[cursor["nth_line"]])-1 == cursor["nth_char"]: #カーソルが行末を指していないかを確認
            return False #行末を指す
        else:
            return True # カーソルの右に文字がある
    elif direction.lower() == "left":
        if cursor["nth_char"] == 0:
            return False #行頭を指す
        else:
            return True #カーソルの左に文字がある
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."


# カーソルの上または下に行が存在するか確認する
def is_there_a_line(direction):
    if direction.lower() == "above":
        if cursor["nth_line"] == 0:
            return False
        else:
            return True
    elif direction.lower() == "below":
        if len(text)-2 == cursor["nth_line"]:
            return False
        else:
            return True
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# カーソルを一つ上か下か右か左へ動かす, 遷移先の存在はチェックはしない。
def move_cursor_by_one_position_to(direction):
    if direction == "above":
        cursor["nth_line"] -= 1
    elif direction == "below":
        cursor["nth_line"] += 1
    elif direction == "left":
        cursor["nth_char"] -= 1
    elif direction == "right":
        cursor["nth_char"] += 1
    else:
        pass
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# カーソルに文字が存在するかチェック
def is_there_a_char_on_cursor():
    cursor_char = text[cursor["nth_line"]][cursor["nth_char"]]
    if cursor_char != "\n" and cursor_char != "EOF":
        return True
    else:
        return False
    assert cursor["nth_line"] >= 0 or cursor["nth_char"] >= 0, "The index refer invalid value."

# テキスト入力を受け付ける関数
def read_until_eof():
    line = ""
    while True:
        line = input()
        if line == "END_OF_TEXT":
            text.append(["EOF"])
            break
        line = list(line) # e.g. ["h","e","l","l","o"]
        line.append("\n") # e.g. ["h","e","l","l","o","\n"]
        text.append(line)

# コマンドを受けとり、それに対応する関数を実行する関数
def execute_command(command):
    if command == "a":
        move_cursor_to_head()
    elif command == "e":
        move_cursor_to_end()
    elif command == "p":
        move_cursor_up_and_to_head()
    elif command == "n":
        move_cursor_down_and_to_head()
    elif command == "f":
        move_cursor_to_right()
    elif command == "b":
        move_cursor_to_left()
    elif command == "d":
        delete_char_on_cursor()
    elif command == "k":
        delete_with_yanking()
    elif command == "y":
        paste_buffer()
    else:
        pass

# 最終的なテキストを表示する関数
def print_text():
    while True:
        line = text.pop(0)
        if not line:
            break
        # print("line in print_text() = {}".format(line[0]))
        if line[0] == "EOF":
            break
        line.pop(-1) # 改行文字を削除
        line = "".join(line)
        print(line)

#--------------------------------------------------------------------------------------------------------

# グローバル変数の定義
buffer = "" # バッファを空に設定する
cursor = {"nth_line": 0, "nth_char": 0} # カーソルの指し示すインデックス, lineは何行目かを、charは何文字目かを指し示す。
text = [] # テキストを二次元リストで管理する

def main():

    # EOFまでのテキストを受ける
    read_until_eof()

    # コマンドを受け取りそれを実行する
    while True:
        command = input()
        if command == "-":
            break
        execute_command(command)

        # print("--------------------")
        # print(text)
        # print("cursor = {}".format(cursor))
        # print("buffer = {}".format(buffer))

    # 結果を表示
    print_text()
if __name__ == "__main__":
    main()

hyo
END_OF_TEXT
k
-

